## Quick sample

In [ ]:
from params_factory import make_params_for_grid
from smm_tools_parallel import grid_moments_parallel  # the parallel-safe function I sent

bounds = {
    'q_0':        (0.5, 0.85),
    'prod_q':     (0.2,  0.6),
    'u_bf_m':     (0.40, 2.50),
    's_job':      (0.2, 0.8),
    'alpha':      (0.10, 1.0),   # if really a share/prob in (0,1)
    'z_corr':     (0.80, 0.99),   # AR(1) corr
    'prod_var_z': (0.10, 0.70),
}

# On Windows/macOS, put the following inside a script guarded by __main__:
# if __name__ == "__main__":
df_grid = grid_moments_parallel(
    p_template=None,         # ignored because we pass a factory
    bounds=bounds,
    n_points=5,
    n_rep=5,
    random_state = 1000,
    sample=10,             # downsample the 5^7 grid
    n_jobs=4,                # adjust to your core count
    p_factory=make_params_for_grid
)
print(df_grid.head())

# Full on calibration

In [ ]:
from smm_tools_parallel import (
    fit_smm_global_percent, report_fit,
    SMMPercentConfig, PARAM_NAMES,
    save_de_result, save_fit_table, export_log_to_csv
)
from params_factory import make_params_coarse, make_params_fine

# bounds for your 7 parameters (edit as needed)
bounds = {
    'q_0':        (0.50, 0.85),
    'prod_q':     (0.20, 0.60),
    'u_bf_m':     (0.40, 2.50),
    's_job':      (0.20, 0.80),
    'alpha':      (0.10, 1.00),   # if really a share/prob in (0,1)
    'z_corr':     (0.80, 0.99),   # AR(1) corr
    'prod_var_z': (0.10, 0.70),
}

# your target data moments (names must match your simulator's moment names)
moms_data = {
    'pr_j2j_an': 0.063,                       # for s_job. but wait: this is YEARLY, not QUARTERLY
    'pr_new_hire': 0.128,                  # for alpha
    'layoffs_share_tercile_0': 0.039,      # for q_0/prod_q #what about this one? in the data it's yearly layoff rate of firms
    # 'layoffs_share_tercile_1': ...
    'layoffs_share_tercile_2': 0.030,      # for q_0/prod_q
    'avg_w_growth_10': 0.33,                # for b
    'sd_dypw': 0.39,                       # for sigma_y
    'autocov_ypw_alt': 0.79,               # for lambda_y
}

# pick any subset to match (defaults to moms_data.keys())
target_keys = list(moms_data.keys())

# --- where to log every evaluation ---
log_db = "runs/quickscan/eval_log.sqlite"

# --- quick coarse config ---
cfg = SMMPercentConfig(n_rep=3, average=False)
quick_overrides = {'tol_simple_model': 1e-4, 'tol_full_model': 1e-4, 'sim_ni': 3000, 'sim_nrep': 3}

res = fit_smm_global_percent(
    p_template=None,
    p_factory=make_params_coarse,
    p_overrides=quick_overrides,
    data_moments=moms_data,
    bounds=bounds,
    target_keys=list(moms_data.keys()),
    cfg=cfg,
    seed=123,
    maxiter=35,
    popsize=6,
    n_jobs=4,
    log_db_path=log_db,     # <-- enable per-evaluation logging
    polish=True,
)

# save the optimizer result
save_de_result(res, "runs/quickscan/de_result")

# verify/present a fit table (e.g., at higher fidelity)
df_fit = report_fit(
    res.x, moms_data, bounds, list(moms_data.keys()),
    cfg=SMMPercentConfig(n_rep=8, average=False),
    p_factory=make_params_fine,
    p_overrides={'tol_simple_model': 1e-6, 'tol_full_model': 1e-6, 'sim_ni': 15000, 'sim_nrep': 8}
)
save_fit_table(df_fit, "runs/quickscan/fit_table.csv")

# export the full evaluation log (all tried θ and their moments/objective)
df_log = export_log_to_csv(log_db, "runs/quickscan/eval_log.csv")
print("Logged rows:", len(df_log))


## Traceback of an error

In [ ]:
import traceback
from smm_tools import simulate_moments_for_params

theta = {
    'q_0': 0.675,
    'prod_q': 0.4,                 # from your warning
    'u_bf_m': 1.45,
    's_job': 0.5,
    'alpha': 0.55,
    'z_corr': 0.895,
    'prod_var_z': 0.4,
}

try:
    simulate_moments_for_params(p, theta, n_rep=2, raise_on_fail=True)
except Exception:
    print("TRACEBACK ↓↓↓")
    print(traceback.format_exc())

c:\Users\zaloilo\Documents\GitHub\Dynamic-Contracting-Multi-worker-Firm\VFI\CRS\smm_tools.py:53: UserWarning: Simulation failed for {'q_0': np.float64(0.675), 'prod_q': np.float64(0.4), 'u_bf_m': np.float64(1.45), 's_job': np.float64(0.5), 'alpha': np.float64(0.55), 'z_corr': np.float64(0.895), 'prod_var_z': np.float64(0.39999999999999997)}: CPUDispatcher(<function Values_int at 0x000001F06BE2F2E0>) returned a result with an exception set
  warnings.warn(f"Simulation failed for {theta_dict}: {e}")
